In [1]:
import numpy as np
import scipy
import pandas as pd

from astropy.table import Table
from astropy import units as u
from astropy import constants as const
from astroquery.gaia import Gaia
from astropy import table

from tqdm import tqdm

import stam

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import ticker
%matplotlib tk

plt.style.use('Nature')

from IPython.display import Image

In [2]:
# Nature standards: https://www.nature.com/nature/for-authors/final-submission
figwidth_single = 89/25.4  # [inch]
figwidth_double = 183/25.4  # [inch]
gaia = Table.read('../data/other/GaiaDR3_100pc_clean.fits')
parsec_imported = False

## Plot CMD

In [3]:
sources = Table.read('../table_cut.fits')
clusters = Table.from_pandas(pd.read_csv('../data/hunt_clusters/clusters.csv'))
members = Table.from_pandas(pd.read_csv('../data/hunt_clusters/members.csv'))

In [4]:
PARSEC_path = '../data/PARSEC v1.2S/Gaia DR3/'
models = stam.getmodels.read_parsec(path=PARSEC_path)
parsec_imported = True

Taking PARSEC files from ../data/PARSEC v1.2S/Gaia DR3/


In [5]:
sources['cst'] = [clusters[clusters['id'] == id]['cst'] for id in sources['id']]
sources['class_50'] = [clusters[clusters['id'] == id]['class_50'] for id in sources['id']]
cut = ~((sources['class_50'] < 0.5) | (sources['cst'] < 5)).flatten()

In [14]:
def plot_cmd(tables,mrkr_colors,markers,labels,title,color_column='bp_rp',magnitude_column='mg',plot_isochrone = False, log_age = [8], mh = [0],a_v = 0,alpha=1):
    # get tables in a list
    # colors and markers as list of strings corresponding to each table
    fig, ax = plt.subplots(figsize=(figwidth_single, figwidth_single), tight_layout=True)   
    ax.hist2d(gaia["bp_rp"], gaia["mg"], bins=500, cmap="Greys", norm=colors.PowerNorm(0.5), zorder=0.5) ## background CMD reference

    if not(parsec_imported):
        print('load parsec isochrones')
    elif plot_isochrone:
        log_age_res = 0.025  # [dex]
        stage_min = 0  # pre-main sequence
        stage_max = 10  # red giant branch
        mass_min = 0  # [Msun]
        mass_max = 10  # [Msun]
        clr = ['k']
        for l,m,c in zip(log_age,mh,clr):
            tracks = stam.gentracks.get_isotrack(models, [l, m], params=("log_age", "mh"), return_table=True,
                                        log_age_res=log_age_res, mass_min=mass_min, mass_max=mass_max,
                                        stage=None, stage_min=stage_min, stage_max=stage_max, sort_by="age", color_filter1="G_BP", color_filter2="G_RP",
                    mag_filter="G")
            x = np.array(tracks["bp_rp"])
            y = np.array(tracks["mg"])
            z = np.array(tracks["mass"])
            ax.plot(x, y, 'o', markersize=0.5,label=f'{float(f"{10**l*1e-6:.2g}"):g} Myr, M/H = {m:.2f}',c=c)

    for tbl,clr,mrkr,lbl,av,al in zip(tables,mrkr_colors,markers,labels,a_v,alpha):
        bp_rp = tbl[color_column] ## before dereddening
        mg = tbl[magnitude_column] ## before dereddening
        # ax.scatter(bp_rp, mg, s=5, c='r') 
        e_bv = av / 3.1
        e_bprp, A_G = stam.gaia.get_extinction_in_band(e_bv,mag_filter="G",color_filter1='G_BP',color_filter2='G_RP')
        bp_rp_corrected = bp_rp - e_bprp
        mg_corrected = mg - A_G
        ax.scatter(bp_rp_corrected, mg_corrected, s=5, marker = mrkr, c=clr,label=lbl,alpha=al)
    ax.invert_yaxis()
    ax.set_xlabel(r"$G_\text{BP} - G_\text{RP}$")
    ax.set_ylabel(r"$G_\text{abs}$")
    ax.set_xlim([-1, 5.5])
    ax.set_ylim([17.6, -2])
    ax.legend()
    ax.set_title(title)
    return fig

In [28]:
i_arr = np.arange(len(sources))[cut]
for i in i_arr:
    src = sources[i]
    nbr = members[members['id'] == src['id']]
    nbr = nbr[nbr['probability'] > 0.99]
    # fld = Table.read(f'../field/field_circ{i+1}.csv')
    
    a_v = src['av_for_mass_interp']
    mh = [src['Fe_H_cluster']]
    log_age = [src['log_age_50']]
    cluster_id = src['id']
    cluster_name = clusters[clusters['id'] == cluster_id]['name'].data[0]
    nbr['mg'] = nbr['phot_g_mean_mag'] +  5 * np.log10(nbr['parallax']) - 10

    tables = [nbr,src]
    mrkr_colors = ['b','r']
    labels = [f'{len(nbr)} cluster members',f'candidate m1={src["m1"]:.2g}/m2={src["m2"]:.2g}']
    # labels = [f'{len(nbr)} cluster members av = {a_v:1.2f}',f'candidate m1={src["parsec_m1"]:1.1f}/m2={src["parsec_m2"]:1.1f} av = {a_v:1.2f}'
    #                         ,f'members av={a_v2:1.2f}', f'candidate av={a_v2:1.2f}']
    markers = ['o','o']
    alpha = [1,1]
    ttl = f'candidate {i} in cluster ' + cluster_name

    fig = plot_cmd(tables,mrkr_colors,markers,labels,ttl,'bp_rp','mg',True,log_age,mh,np.array([a_v,a_v]),alpha)
    fig.savefig(f'../img/cmd/cmd{i}.png',dpi=300)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_22480\2753199505.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(figwidth_single, figwidth_single), tight_layout=True)


## Create auxiliary table

In [22]:
sources = Table.read('../mmt_hectoechelle_extra.csv')
clusters = Table.from_pandas(pd.read_csv('../data/hunt_clusters/clusters.csv'))
members = Table.from_pandas(pd.read_csv('../data/hunt_clusters/members.csv'))

In [12]:
from astropy.coordinates import SkyCoord
Gaia.ROW_LIMIT = 1000000

def create_neighbors_field_files(i,ang_radius):
    ra = sources[i]['ra']
    dec = sources[i]['dec']
    cluster_id = sources[i]['id']
    neighbors = members[members['id'] == cluster_id]
    idlst = neighbors['source_id'].data
    
    query = '''SELECT source_id, astrometric_excess_noise, astrometric_excess_noise_sig,
    astrometric_params_solved, astrometric_sigma5d_max,
     ruwe, radial_velocity, radial_velocity_error ,rv_method_used ,teff_gspphot, teff_gspphot_lower,
         teff_gspphot_upper, logg_gspphot, logg_gspphot_lower, logg_gspphot_upper, ag_gspphot, ag_gspphot_lower,ag_gspphot_upper
        FROM gaiadr3.gaia_source
        WHERE source_id IN {id_lst}
        '''.format(id_lst = tuple(neighbors['source_id'].data))
    job = Gaia.launch_job(query= query)
    gaiaextra = job.get_results()

    pmra_lo = sources[i]['pmra_cluster'] - 3*sources[i]['pmra_std_cluster']
    pmra_hi = sources[i]['pmra_cluster'] + 3*sources[i]['pmra_std_cluster']
    pmdec_lo = sources[i]['pmdec_cluster'] - 3*sources[i]['pmdec_std_cluster']
    pmdec_hi = sources[i]['pmdec_cluster'] + 3*sources[i]['pmdec_std_cluster']
    parallax_lo = sources[i]['parallax_cluster'] - 3*sources[i]['parallax_std_cluster']
    parallax_hi = sources[i]['parallax_cluster'] + 3*sources[i]['parallax_std_cluster']

    cntr = SkyCoord(sources[i]['ra_cluster'],sources[i]['dec_cluster'],unit='deg',frame='icrs')

    fieldtbl = Gaia.cone_search(cntr,radius = ang_radius*u.deg,table_name='gaiadr3.gaia_source',columns =['source_id', 'ra','dec','parallax','parallax_error',
    'bp_rp','phot_g_mean_mag','pmra','pmra_error','pmdec','pmdec_error', 'astrometric_excess_noise', 'astrometric_excess_noise_sig',
    'astrometric_params_solved', 'astrometric_sigma5d_max', 'ruwe','radial_velocity', 
    'radial_velocity_error' ,'rv_method_used' ,'teff_gspphot', 'teff_gspphot_lower',
      'teff_gspphot_upper', 'logg_gspphot', 'logg_gspphot_lower', 'logg_gspphot_upper', 'ag_gspphot', 'ag_gspphot_lower','ag_gspphot_upper']).get_results()
    cond1 =  fieldtbl['pmra'] < pmra_hi
    cond2 = fieldtbl['pmra'] > pmra_lo
    cond3 = fieldtbl['pmdec'] < pmdec_hi
    cond4 = fieldtbl['pmdec'] > pmdec_lo 
    cond5 = fieldtbl['parallax'] < parallax_hi
    cond6 = fieldtbl['parallax'] > parallax_lo 
    cond7 = np.isin(fieldtbl['source_id'],idlst) 
    cond = cond1 & cond2 & cond3 & cond4 & cond5 & cond6 &  ~(cond7)
    fieldtbl  = fieldtbl[cond]
  
    # neighbors['mg'] = neighbors['phot_g_mean_mag'] + 5 * np.log10(neighbors['parallax']) - 10
    # neighbors.keep_columns(['source_id','ra','dec','parallax','parallax_error','mg','bp_rp','id','name','probability','phot_g_mean_mag','pmra','pmra_error','pmdec','pmdec_error'])
    # neighbors = table.join(neighbors,gaiaextra,keys=['source_id'],join_type='left')
    # neighbors.to_pandas().to_csv(f'../folder/cluster_neighbors{i+1}.csv')

    fieldtbl['mg'] = fieldtbl['phot_g_mean_mag'] + 5 * np.log10(fieldtbl['parallax']) - 10
    fieldtbl.sort('mg')
    # if(len(fieldtbl) > 200):
    #     fieldtbl = fieldtbl[:200]
    fieldtbl.to_pandas().to_csv(f'../folder/fullfield{i+1}.csv')
    print(f'{len(neighbors)} neighbors')
    print(f'{len(fieldtbl)} field stars')